In [61]:
import pandas as pd
import math
import numpy as np
import requests
import time
from collections import Counter
from sklearn.model_selection import train_test_split

In [62]:
df = pd.read_excel('./data.xlsx', index_col=0)
df.columns

Index(['pair', 'token_name', 'token_address', 'price_b', 'price_s',
       'token_age_b', 'token_age_s', 'transfers', 'buys_m5', 'sells_m5',
       'buys_h1', 'sells_h1', 'buys_h6', 'sells_h6', 'buys_h24', 'sells_h24',
       'volume_m5', 'volume_h1', 'volume_h6', 'volume_h24', 'price_change_m5',
       'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
       'fdv', 'market_cap', 'is_mutable_metadata', 'is_telegram', 'is_twitter',
       'is_website', 'boosts', 'telegram_members', 'is_telegram_error',
       'twitter_days', 'twitter_tweets', 'twitter_followers',
       'twitter_smart_followers', 'is_twitter_error', 'opening_date',
       'closing_date', 'sns_bought', 'sns_sold', 'sns_held_all',
       'sns_sold_some', 'sns_sold_all', 'tt_bought', 'tt_sold', 'PNL',
       'PNL_10', 'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50', 'PNL_loss_5',
       'PNL_loss_10', 'status', 'price_change_check', 'mode'],
      dtype='object')

In [63]:
drop_columns = ['pair', 'token_name',
    'buys_h6', 'sells_h6', 'buys_h24', 'sells_h24',
    'volume_h6', 'volume_h24', 'fdv',
    'price_s', 
    #'token_age_s', 
    'opening_date', 
    #'closing_date',
    'status', 'mode']
df = df.drop(drop_columns, axis=1)
df.head()

,token_address,price_b,token_age_b,token_age_s,transfers,buys_m5,sells_m5,buys_h1,sells_h1,volume_m5,...,tt_sold,PNL,PNL_10,PNL_20,PNL_30,PNL_40,PNL_50,PNL_loss_5,PNL_loss_10,price_change_check
id,,,,,,,,,,,,,,,,,,,,,
639,6gSXeE5sqYt9GrBRG6yy9s9ayWhuWL55GtvPSncTpump,"0,000514","30,9","35,87",NaN,411,212,2347,1196,"87364,59",...,1950.0 1809.0 1530.0 2653.0 1364.0 1354.0 1242...,"61,19",1.0,1.0,1.0,1.0,1.0,NaN,NaN,"-5,410379094589613"
636,2CYiGdZ5SUvD4gQQ3arVNJGF1r7BcyfVWHo24ViNpump,"0,0007677","30,93","47,42",NaN,183,183,1349,984,"50024,71",...,24200.0 5612.0 999.0 1569.0 1564.0 755.0 1468....,"-35,44",1.0,1.0,NaN,NaN,NaN,1.0,1.0,"3,6585365853658582"
634,FV5jxCRt66m8rSnpYDm8SRvhUe3Q8BTcRT67ihBspump,"0,001264","9,65","30,36",NaN,1108,410,1499,628,"590851,75",...,25500.0 10400.0 10000.0 9727.0 8873.0 7924.0 7...,"-31,80",1.0,1.0,1.0,1.0,NaN,1.0,1.0,"39,63764913831197"
633,3n5AnsUk1bRAUC1PAH4xYDMqXRsK1fDAkAfuX5ippump,"0,0004653","176,93","202,67",NaN,49,36,1022,868,"26633,48",...,7253.0 2095.0 1593.0 1654.0 2268.0 699.0 685.0...,"60,82",1.0,1.0,1.0,1.0,1.0,NaN,NaN,"7,509242144177445"
632,G3tYkTEg2jVq7RhDgLTgTMqQ17qAcT3dMKpUurBcpump,"0,000156","11,93","27,74",NaN,176,97,576,263,"33261,38",...,5247.0 4224.0 3727.0 3365.0 979.0 962.0 956.0 ...,"-31,92",1.0,1.0,1.0,1.0,NaN,1.0,1.0,"11,98851399856425"


In [64]:
#df = df.dropna(subset=['transfers', 'PNL'])
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, 639 to 110
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   token_address            505 non-null    object 
 1   price_b                  505 non-null    object 
 2   token_age_b              505 non-null    object 
 3   token_age_s              505 non-null    object 
 4   transfers                505 non-null    float64
 5   buys_m5                  505 non-null    int64  
 6   sells_m5                 505 non-null    int64  
 7   buys_h1                  505 non-null    int64  
 8   sells_h1                 505 non-null    int64  
 9   volume_m5                505 non-null    object 
 10  volume_h1                505 non-null    object 
 11  price_change_m5          505 non-null    object 
 12  price_change_h1          505 non-null    object 
 13  price_change_h6          505 non-null    object 
 14  price_change_h24         505 

In [65]:
float_types = [
    'token_age_b', 'token_age_s', 'price_b', 
    'transfers', 'buys_m5', 'sells_m5',
    'buys_h1', 'sells_h1', 
    'volume_m5', 'volume_h1', 'price_change_m5',
    'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
    'market_cap', 'PNL', 'PNL_10', 'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50',
    'price_change_check'
],

int_types = [
    'is_mutable_metadata', 'is_telegram', 'is_twitter',
    'is_website', 'boosts', 'telegram_members', 'is_telegram_error',
    'twitter_days', 'twitter_tweets', 'twitter_followers',
    'twitter_smart_followers', 'is_twitter_error',
    'sns_held_all', 'sns_sold_some', 'sns_sold_all',
]

for col in float_types:
    df[col] = df[col].replace(",",".", regex=True).astype(float)
    
for col in int_types:
    df[col] = df[col].replace(",",".", regex=True).astype(int)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, 639 to 110
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   token_address            505 non-null    object 
 1   price_b                  505 non-null    float64
 2   token_age_b              505 non-null    float64
 3   token_age_s              505 non-null    float64
 4   transfers                505 non-null    float64
 5   buys_m5                  505 non-null    float64
 6   sells_m5                 505 non-null    float64
 7   buys_h1                  505 non-null    float64
 8   sells_h1                 505 non-null    float64
 9   volume_m5                505 non-null    float64
 10  volume_h1                505 non-null    float64
 11  price_change_m5          505 non-null    float64
 12  price_change_h1          505 non-null    float64
 13  price_change_h6          505 non-null    float64
 14  price_change_h24         505 

In [67]:
#df = df[df['is_telegram_error'] == 0]
#df = df[df['is_twitter_error'] == 0]
#df = df[df['price_change_m5'] > -10]
#df = df[df['price_change_h1'] > 10]
#df = df[(df['transfers'] > 100) & (df['transfers'] < 5000)]

In [68]:
df = df.drop(['is_telegram_error', 'is_twitter_error'], axis=1)

In [69]:
df = df.dropna(subset=['sns_bought', 'sns_sold', 'tt_bought', 'tt_sold'])

In [70]:
df = df[df['sns_bought'] != 0]
df = df[df['sns_sold'] != 0]
df = df[df['tt_bought'] != 0]
df = df[df['tt_sold'] != 0]

In [71]:
def covert_to_list(df, col_name):
    all_lsts = []
    for value_str in df[col_name]:
        lst = [float(x) for x in value_str.split(" ")]
        all_lsts.append(lst)
        
    return all_lsts

In [72]:
df['sns_bought_lst'] = covert_to_list(df, 'sns_bought')
df['sns_sold_lst'] = covert_to_list(df, 'sns_sold')
df['tt_bought_lst'] = covert_to_list(df, 'tt_bought')
df['tt_sold_lst'] = covert_to_list(df, 'tt_sold')

In [73]:
# from collections import Counter

# def get_most_common(df, col_name, is_zero=True):
#     result = []
#     for lst in df[col_name]:
#         copy_lst = lst.copy()
#         if not is_zero:
#             if 0.0 in copy_lst:
#                 copy_lst = list(
#                     filter(
#                         lambda x: x != 0.0,
#                         lst
#                     )
#                 )
#         if copy_lst:
#             occurence_count = Counter(copy_lst)
#             result.append(occurence_count.most_common(1)[0])
#         else:
#             result.append((0.0, 0.0))
#         #print(occurence_count.most_common(1)[0][0])
#     return result
    
# df['sns_most_common'] = [x[0] for x in get_most_common(df, 'sns_bought_lst')]
# df['sns_most_common_except_0'] = [x[0] for x in get_most_common(df, 'sns_bought_lst', False)]

In [74]:
def sum_values(df: pd.DataFrame, col_name: str, num: int) -> pd.DataFrame:
    new_values_lst = []
    for lst in df[col_name]:
        end = len(lst) + 1 if len(lst) < num else num
        result = 0
        for value in lst[:end]:
            result += value
        new_values_lst.append(result)
        
    return new_values_lst

In [75]:
df['tt_f10_bought']= sum_values(df, 'tt_bought_lst', 100)
df['tt_f10_sold'] = sum_values(df, 'tt_sold_lst', 100)
df['sns_f10_bought'] = sum_values(df, 'sns_bought_lst', 100)
df['sns_f10_sold'] = sum_values(df, 'sns_sold_lst', 100)

In [76]:
def get_pnl(df: pd.DataFrame, bought_col: str, sold_col: str) -> list[list]:
    all_bought = []
    for value_str in df[bought_col]:
        bought_lst = [float(x) for x in value_str.split(" ")]
        all_bought.append(bought_lst)
        
    all_sold = []
    for value_str in df[sold_col]:
        sold_lst = [float(x) for x in value_str.split(" ")]
        all_sold.append(sold_lst)
    
    all_pnl = []
    for each_bought, each_sold in zip(all_bought, all_sold):
        each_pnl = [sold - bought if sold else 0 for bought, sold in zip(each_bought, each_sold)]
        all_pnl.append(each_pnl)
    
    return all_pnl

In [77]:
df['sns_pnl'] = get_pnl(df, 'sns_bought', 'sns_sold')
df['tt_pnl'] = get_pnl(df, 'tt_bought', 'tt_sold')

In [78]:
def count_zero(df: pd.DataFrame, col_name: str, amount: int | None=None):
    count_zero = []
    for lst in df[col_name]:
        count_zero.append(lst.count(0))
    
    return count_zero

In [79]:
df['sns_no_bought'] = count_zero(df, 'sns_bought_lst')
df['sns_no_sold'] = count_zero(df, 'sns_sold_lst')
df['tt_no_bought'] = count_zero(df, 'tt_bought_lst')
df['tt_no_sold'] = count_zero(df, 'tt_sold_lst')

In [80]:
def count_pnl_loss(df: pd.DataFrame, col_name: str):
    count_pnl_loss = []
    for pnl_lst in df[col_name]:
        pnl_loss = sum(i < 0 for i in pnl_lst)
        count_pnl_loss.append(pnl_loss)
        
    return count_pnl_loss

In [81]:
df['sns_pnl_loss'] = count_pnl_loss(df, 'sns_pnl')
df['tt_pnl_loss'] = count_pnl_loss(df, 'tt_pnl')

In [82]:
df = df.drop(
    [
        'tt_bought', 'tt_sold', 
        'sns_bought', 'sns_sold', 
        'tt_bought_lst', 'tt_sold_lst', 
        'sns_bought_lst', 'sns_sold_lst', 
        'sns_pnl', 'tt_pnl', 
    ], 
    axis=1
)

In [83]:
def get_pairs_count(token_address):
    time.sleep(1)
    try:
        pairs = requests.get("https://api.dexscreener.com/latest/dex/tokens/" + token_address).json()["pairs"]
        count = len(pairs)
    except:
        count = 0
    return count

In [84]:
df = df.drop(["token_address", "transfers", "is_mutable_metadata"], axis=1)

In [85]:
# result_df = df.loc[
#     (df['sells_m5'] < 700) 
#     & (df['sells_h1'] < 4000) 
#     #& (df['market_cap'] < 500000) 
#     #& (df['boosts'] == 500) 
#     #& (df['sns_held_all'] > 40) 
#     & (df['sns_no_bought'] < 20) 
#     #& (df['tt_no_sold'] > 0) 
#     #& (df['buys_h1'] < 10000) 
#     # & (df['token_age_b'] >= 10)
#     # & (df['token_age_b'] <= 120)
#     #& (df['price_b'] >= 0.00001)
#     #& (df['telegram_members'] <= 1000)
#     # & (df['sns_pnl_loss'] <= 20)
#     # & (df['tt_no_bought'] <= 50)
#     & (df['tt_pnl_loss'] <= 20) # yes
#     & (df['price_change_m5'] >= -4)
#     & (df['price_change_m5'] <= 60)
#     #& (df['tt_f10_sold'] / df['tt_f10_bought'] <= 60)
# ]

# # profit_df = result_df.loc[df['PNL'] >= 60]
# # loss_df = result_df.loc[df['PNL'] < 60]

# take_profit = 1.4
# stop_loss = 0.8

# profit_df = result_df.loc[df['PNL_40'] == 1] 
# loss_df = result_df.loc[df['PNL_40'] == 0] 
# profit_count = profit_df.shape[0]
# loss_cout = loss_df.shape[0]

# print(f"Выигрыш: {profit_count}")
# print(f"Проигрыш: {loss_cout}")
# success = profit_count / (profit_count + loss_cout) * 100
# print(f"Процент выигрышей: {success:.2f} %")

# sum_buy = 10
# money = (sum_buy * take_profit - sum_buy) * profit_count - (sum_buy - sum_buy * stop_loss) * loss_cout
# print(f"Заработано {money} $")

In [86]:
result_df = df.loc[
    (df['sells_m5'] < 400) 
    & (df['sells_h1'] < 1000) 
    & (df['market_cap'] < 500000) 
    & (df['boosts'] == 500) 
    #& (df['sns_held_all'] > 40) 
    #& (df['sns_no_bought'] < 20) 
    #& (df['tt_no_sold'] > 0) 
    #& (df['buys_h1'] < 10000) 
    & (df['token_age_b'] >= 10)
    & (df['token_age_b'] <= 120)
    #& (df['price_b'] >= 0.00001)
    #& (df['telegram_members'] <= 1000)
    & (df['sns_pnl_loss'] <= 20)
    & (df['tt_no_bought'] <= 50)
    & (df['tt_pnl_loss'] <= 20) # yes
    & (df['price_change_m5'] <= 60)
    & (df['tt_f10_bought'] <= 40000)
    #& (df['tt_f10_sold'] / df['tt_f10_bought'] <= 60)
]

# profit_df = result_df.loc[df['PNL'] >= 60]
# loss_df = result_df.loc[df['PNL'] < 60]

take_profit = 1.5
stop_loss = 0.8

profit_df = result_df.loc[df['PNL_20'] == 1] 
loss_df = result_df.loc[df['PNL_20'] == 0] 
profit_count = profit_df.shape[0]
loss_cout = loss_df.shape[0]

print(f"Выигрыш: {profit_count}")
print(f"Проигрыш: {loss_cout}")
success = profit_count / (profit_count + loss_cout) * 100
print(f"Процент выигрышей: {success:.2f} %")

sum_buy = 10
money = (sum_buy * take_profit - sum_buy) * profit_count - (sum_buy - sum_buy * stop_loss) * loss_cout
print(f"Заработано {money} $")

Выигрыш: 19
Проигрыш: 3
Процент выигрышей: 86.36 %
Заработано 89.0 $


In [87]:
# result_df = df.loc[
#     (df['sells_m5'] < 400000) 
#     # & (df['sells_h1'] < 1000) 
#     # & (df['market_cap'] < 500000) 
#     #& (df['price_change_check'] >= 0)
#     # & (df['price_change_check'] <= 30)
#     #& (df['boosts'] == 500) 
#     #& (df['sns_held_all'] > 40) 
#     #& (df['sns_no_bought'] < 20) 
#     & (df['tt_no_bought'] <= 10) 
#     #& (df['buys_h1'] < 10000) 
#     #& (df['token_age_b'] >= 5)
#     #& (df['token_age_b'] <= 3600)
#     #& (df['price_b'] >= 0.00001)
#     #& (df['telegram_members'] <= 1000)
#     # & (df['sns_pnl_loss'] <= 20)
#     & (df['tt_pnl_loss'] <= 20) # yes
#     # & (df['price_change_m5'] <= 60)
#     #& (df['tt_f10_sold'] / df['tt_f10_bought'] <= 60)
# ]

# # profit_df = result_df.loc[df['PNL'] >= 60]
# # loss_df = result_df.loc[df['PNL'] < 60]

# take_profit = 1.2
# stop_loss = 0.8

# profit_df = result_df.loc[df['PNL_20'] == 1] 
# loss_df = result_df.loc[df['PNL_20'] == 0] 
# profit_count = profit_df.shape[0]
# loss_cout = loss_df.shape[0]

# print(f"Выигрыш: {profit_count}")
# print(f"Проигрыш: {loss_cout}")
# success = profit_count / (profit_count + loss_cout) * 100
# print(f"Процент выигрышей: {success:.2f} %")

# sum_buy = 10
# money = (sum_buy * take_profit - sum_buy) * profit_count - (sum_buy - sum_buy * stop_loss) * loss_cout
# print(f"Заработано {money} $")

In [88]:
df.to_csv('./clean_data.csv', index=False)

In [89]:
result_df.to_excel('./clean_data.xlsx', index=False)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 473 entries, 639 to 110
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   price_b                  473 non-null    float64
 1   token_age_b              473 non-null    float64
 2   token_age_s              473 non-null    float64
 3   buys_m5                  473 non-null    float64
 4   sells_m5                 473 non-null    float64
 5   buys_h1                  473 non-null    float64
 6   sells_h1                 473 non-null    float64
 7   volume_m5                473 non-null    float64
 8   volume_h1                473 non-null    float64
 9   price_change_m5          473 non-null    float64
 10  price_change_h1          473 non-null    float64
 11  price_change_h6          473 non-null    float64
 12  price_change_h24         473 non-null    float64
 13  liquidity                473 non-null    float64
 14  market_cap               473 